In [1]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
    !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
    !git fetch --all
    !git reset --hard origin/master

In [2]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
#     !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
#     !git pull

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import tradestats
import pandas as pd
from datetime import datetime

In [5]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
import graphviz

In [6]:
importlib.reload(tradestats)
from tradestats import *

In [7]:
prices = loaddata_4h('AUDUSD_Candlestick_4_Hour_BID_03.08.2003-04.09.2021_EEF_GMT.csv')

In [8]:
rsiKey = [13]
tdiKey = [13]
dictList = []
colFeat = []
keyList = []
# rsiDict   = rsi(prices, rsiKey)
tdiDict   = tdi1(prices, tdiKey)
# dictList.append(rsiDict.df); colFeat.append('rsi'); keyList.append(rsiKey) 
dictList.append(tdiDict.df); colFeat.append('tdi'); keyList.append(tdiKey) 

In [9]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
masterFrame[['id','year','month','day','date','hour','weekday','open','high','low','close','volume']] = prices[['id','year','month','day','date','hour','weekday','open','high','low','close','volume']]   

In [10]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [11]:
# masterFrame.to_csv(sep=';',path_or_buf='../Data/masterFrame.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [12]:
masterFrame = masterFrame.dropna()

In [13]:
# masterFrame.iloc[0:-50,:20]

In [104]:
importlib.reload(tradestats)
from tradestats import *
starttime = datetime.now()
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.003],False)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5,9],[13,17],[0.001,0.002,0.003,0.004,0.005,0.006],True)
alltrades = preparetrades(masterFrame,[1],[5],[13],[0.001,0.002,0.003,0.004,0.005,0.006],2015,2021)
trades = cleartrades(alltrades.copy(),True)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.002],True)
endtime = datetime.now()
print(str(endtime - starttime))

0:00:07.873137


In [103]:
#____________________________________________________________________________________________________________________
#
#
trades.head(2)

,year,month,hour,tdi13rsi1,tdi13red1,tdi13green1,tdi13mid1,tdi13green1_red1,tdi13red2,tdi13green2,...,tdi13mid_slope,tdi13habarsize1,tdi13hacolor1,tdi13barnumber1,tdi13habarsize2,tradetype,sl,closehour,profit,profit1
18473,2015,1,5,50.698531,54.161562,50.698531,54.236735,-3.463031,56.012807,50.698531,...,0.181345,0.0000,1,2,0.0000,1,10.0,13,0.0,-1
18479,2015,1,5,39.628409,45.833649,39.628409,53.960233,-6.205240,47.415095,39.206507,...,-0.296150,1.8875,1,1,-23.8125,1,10.0,13,-10.0,-1


In [ ]:
X = trades.iloc[:, :-1].values
feature_names = trades.columns[:-1]
y = trades.iloc[:, -1:].values

In [ ]:
# regr = DecisionTreeRegressor(min_samples_leaf=0.001,criterion='mse',max_depth=17, random_state=1234)
# criterion{“mse”, “friedman_mse”, “mae”, “poisson”}, default=”mse”
regr = DecisionTreeClassifier(min_samples_leaf=1,criterion='gini',max_depth=8, random_state=1234)
# criterion{“gini”, “entropy”}, default=”gini”
regr.fit(X, y)
score = regr.score(X, y)
print(score)

In [ ]:
"""
all features mse depth 8 leaf 0.05 =0,0049
wyciąłem poniższe mse depth 8 leaf 0.05 =0,0043
#     df['green1_red1'] = df.green1 - df.red1
#     df['green2_red2'] = df.green2 - df.red2
#     df['green_red_change'] = df.green1_red1 - df.green2_red2
#     df['green_red_slope_change'] = df.green_slope - df.red_slope
all features mse depth 8 leaf 0.1 =0.0028
all features friedman_mse depth 8 leaf 0.05 =0.0049
all features mse depth 8 leaf 0.03 = 0.0069
all features mse depth 8 leaf 0.01 = 0.0074
all features mse depth 8 leaf 1 = 0.048
all features mse depth 9 leaf 1 = 0.051
all features mse depth 15 leaf 1 = 0.11
all features mse depth 15 leaf 0.001 = 0.054
all features mse depth 15 leaf 0.05 = 0,028
all features mse depth 5 leaf 0.05 = 0.0016
"""

In [ ]:
dd = pd.DataFrame()
dd['f'] = feature_names.values
dd['i'] = regr.feature_importances_
dd.sort_values(by=['i'], ascending=False)

In [ ]:
# text_representation = tree.export_text(regr)
# print(text_representation)

In [ ]:
from graphviz import Source
from sklearn import tree
graph = Source( tree.export_graphviz(regr, out_file=None, feature_names=feature_names,rounded = True, proportion = False, 
                 filled = True))
graph.format = 'png'
graph.render('dtree_render',view=True)

In [ ]:
alltrades.head()

In [15]:
importlib.reload(tradestats)
from tradestats import *
stats = stattrades(alltrades)

In [ ]:
stats.sort_values("profit_ratio",ascending=False)

In [ ]:
importlib.reload(tradestats)
from tradestats import *

tradetypes = [1]
openhours = [5]
closehours = [13]
sls = [10]
bar2froms = [-1000,-20,-10,0,10,20,1000]
bar2tos = [-1000,-20,-10,0,10,20,1000]
bar1froms = [-1000,-20,-10,0,10,20,1000]
bar1tos =[-1000,-20,-10,0,10,20,1000]
bar2froms = [-1000,1000]
bar2tos = [-1000,1000]
bar1froms = [-1000,1000]
bar1tos =[-1000,1000]
gr2froms = [-1000,-8,-3,0,3,8,1000]
gr2tos = [-1000,-8,-3,0,3,8,1000]
gr1froms = [-1000,-8,-3,0,3,8,1000]
gr1tos = [-1000,-8,-3,0,3,8,1000]
gr2froms = [-1000,-10,-8,-6,-4,-2,0,2,4,6,8,10,1000]
gr2tos = [-1000,-10,-8,-6,-4,-2,0,2,4,6,8,10,1000]
gr1froms = [-1000,-10,-8,-6,-4,-2,0,2,4,6,8,10,1000]
gr1tos = [-1000,-10,-8,-6,-4,-2,0,2,4,6,8,10,1000]
starttime = datetime.now()
stats = stathyper(alltrades,tradetypes,openhours,closehours,sls,bar2froms,bar2tos,bar1froms,bar1tos,gr2froms,gr2tos,gr1froms,gr1tos)
endtime = datetime.now()
print(str(endtime - starttime))

In [ ]:
importlib.reload(tradestats)
from tradestats import *

params = {}

params['tradetypes'] = [1]
params['openhours'] = [5]
params['closehours'] = [13]
params['sls'] = [10]

params['bar2froms'] = [-1000,-8,0,8,1000]
params['bar2tos'] = [-1000,-8,0,8,1000]
params['bar2froms'] = [-1000]
params['bar2tos'] = [1000]

params['bar1froms'] = [-1000,-8,0,8,1000]
params['bar1tos'] =[-1000,-8,0,8,1000]
# params['bar1froms'] = [-1000]
# params['bar1tos'] =[1000]

params['gr2froms'] = [-1000]
params['gr2tos'] = [1000]

params['gr1froms'] = [-1000,0,1000]
params['gr1tos'] = [-1000,0,1000]
# params['gr1froms'] = [-1000]
# params['gr1tos'] = [1000]

params['rslopefroms'] = [-1000,-3,0,3,1000]
params['rslopetos'] = [-1000,-3,0,3,1000]

params['gslopefroms'] = [-1000,-8,-3,0,3,8,1000]
params['gslopetos'] = [-1000,-8,-3,0,3,8,1000]

params['grcfroms'] = [-1000]
params['grctos'] = [1000]

params['redfroms'] = [0,40,60,100]
params['redtos'] = [0,40,60,100]
# params['redfroms'] = [0,100]
# params['redtos'] = [0,100]

params['barnofroms'] = [1]
params['barnotos'] = [1,2,3,100]
# params['barnofroms'] = [1,100]
# params['barnotos'] = [1,100]

params['crossfroms'] = [0,1]
params['crosstos'] = [0,1]

params['filename'] = '2015_2021_5_13_10'

stats = stathyperparams(alltrades,params)

In [ ]:
#######TEST########
importlib.reload(tradestats)
from tradestats import *

params = {}

params['tradetypes'] = [1]
params['openhours'] = [5]
params['closehours'] = [13]
params['sls'] = [10]

params['bar2froms'] = [-1000]
params['bar2tos'] = [1000]

params['bar1froms'] = [-1000]
params['bar1tos'] =[1000]

params['gr2froms'] = [-1000]
params['gr2tos'] = [1000]

params['gr1froms'] = [-1000]
params['gr1tos'] = [1000]

params['rslopefroms'] = [-1000,-3,0,3,1000]
params['rslopetos'] = [-1000,-3,0,3,1000]

params['gslopefroms'] = [-1000,-8,-3,0,3,8,1000]
params['gslopetos'] = [-1000,-8,-3,0,3,8,1000]

params['grcfroms'] = [-1000]
params['grctos'] = [1000]

params['redfroms'] = [0,100]
params['redtos'] = [0,100]

params['barnofroms'] = [1,100]
params['barnotos'] = [1,100]

params['crossfroms'] = [0,1]
params['crosstos'] = [0,1]

params['filename'] = '2015_2021_5_13_10'

stats = stathyperparams(alltrades,params)

In [ ]:
stats.sort_values("profit_sum",ascending=False)

In [ ]:
stats.to_csv(sep=';',path_or_buf='../Data/stats0.csv',date_format="%Y-%m-%d",index = False,na_rep='')